# House price dataset preparation

In [1]:
import numpy as np
import pandas as pd

In [2]:
# load dataset
data = pd.read_csv('../houseprice.csv')

# drop id
data.drop(columns=["Id"], inplace=True)

# cast as object
data['MSSubClass'] = data['MSSubClass'].astype('O')

# rows and columns of the data
print(data.shape)

# visualise the dataset
data.head()

(1460, 80)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
# impute categorical variables

cat_vars = [var for var in data.columns if data[var].dtype == 'O']

data[cat_vars] = data[cat_vars].fillna('Missing')

In [4]:
# impute numerical variables

num_vars = [
    var for var in data.columns if var not in cat_vars + ['SalePrice'] and
    data[var].isnull().sum() > 0
]

mean_dict = data[num_vars].mean().to_dict()

data = data.fillna(mean_dict)

In [5]:
# capture time since

for var in ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']:
    data[var] = data['YrSold'] - data[var]
    
data.drop(['YrSold'], axis=1, inplace=True)

In [6]:
# binarize skewed variables

skewed = [
    'BsmtFinSF2', 'LowQualFinSF', 'EnclosedPorch',
    '3SsnPorch', 'ScreenPorch', 'MiscVal'
]

for var in skewed:
    data[var] = np.where(data[var]==0, 0, 1)

In [7]:
# re-map strings to numbers, which determine quality

qual_mappings = {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5, 'Missing': 0, 'NA': 0}

qual_vars = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
             'HeatingQC', 'KitchenQual', 'FireplaceQu',
             'GarageQual', 'GarageCond',
            ]

for var in qual_vars:
    data[var] = data[var].map(qual_mappings)

In [8]:
exposure_mappings = {'No': 1, 'Mn': 2, 'Av': 3, 'Gd': 4, "Missing": 0}

var = 'BsmtExposure'

data[var] = data[var].map(exposure_mappings)

In [9]:
finish_mappings = {'Missing': 0, 'NA': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6}

finish_vars = ['BsmtFinType1', 'BsmtFinType2']

for var in finish_vars:
    data[var] = data[var].map(finish_mappings)

In [10]:
garage_mappings = {'Missing': 0, 'NA': 0, 'Unf': 1, 'RFn': 2, 'Fin': 3}

var = 'GarageFinish'

data[var] = data[var].map(garage_mappings)

In [11]:
fence_mappings = {'Missing': 0, 'NA': 0, 'MnWw': 1, 'GdWo': 2, 'MnPrv': 3, 'GdPrv': 4}

var = 'Fence'

data[var] = data[var].map(fence_mappings)

In [12]:
# capture all quality variables

qual_vars  = qual_vars + finish_vars + ['BsmtExposure','GarageFinish','Fence']

# capture the remaining categorical variables
# (those that we did not re-map)

cat_others = [
    var for var in cat_vars if var not in qual_vars
]

len(cat_others)

30

In [13]:
# remove rare categories

def find_frequent_labels(df, var, rare_perc):
    
    # function finds the labels that are shared by more than
    # a certain % of the houses in the dataset

    df = df.copy()

    tmp = df.groupby(var)[var].count() / len(df)

    return tmp[tmp > rare_perc].index


for var in cat_others:
    
    # find the frequent categories
    frequent_ls = find_frequent_labels(data, var, 0.01)
    
    # replace rare categories by the string "Rare"
    data[var] = np.where(data[var].isin(
        frequent_ls), data[var], 'Rare')

In [14]:
# one hot encoding

data = pd.concat([
    data,
    pd.get_dummies(data[cat_others]).astype(int)],
    axis=1,
)

data.drop(columns=cat_others, inplace=True)

data.head()

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,...,MiscFeature_Shed,SaleType_COD,SaleType_New,SaleType_Rare,SaleType_WD,SaleCondition_Abnorml,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_Rare
0,65.0,8450,7,5,5,5,196.0,4,3,4,...,0,0,0,0,1,0,0,1,0,0
1,80.0,9600,6,8,31,31,0.0,3,3,4,...,0,0,0,0,1,0,0,1,0,0
2,68.0,11250,7,5,7,6,162.0,4,3,4,...,0,0,0,0,1,0,0,1,0,0
3,60.0,9550,7,5,91,36,0.0,3,3,3,...,0,0,0,0,1,1,0,0,0,0
4,84.0,14260,8,5,8,8,350.0,4,3,4,...,0,0,0,0,1,0,0,1,0,0


In [15]:
# check absence of na in the data set
[var for var in data.columns if data[var].isnull().sum() > 0]

[]

In [16]:
data.to_csv("../houseprice_prep.csv", index=False)

That concludes the feature engineering section.

# Additional Resources

- [Feature Engineering for Machine Learning](https://www.trainindata.com/p/feature-engineering-for-machine-learning) - Online Course
- [Packt Feature Engineering Cookbook](https://www.amazon.com/Python-Feature-Engineering-Cookbook-transforming-dp-1804611301/dp/1804611301) - Book
- [Feature Engineering for Machine Learning: A comprehensive Overview](https://www.blog.trainindata.com/feature-engineering-for-machine-learning/) - Article
- [Practical Code Implementations of Feature Engineering for Machine Learning with Python](https://towardsdatascience.com/practical-code-implementations-of-feature-engineering-for-machine-learning-with-python-f13b953d4bcd) - Article